In [1]:
import numpy as np
import pyvista as pv
import open3d
import modeling
from modeling import *
import scipy.ndimage
#import scipy.stats
import copy
from utils import *
from matplotlib import pyplot as plt
from matplotlib import cm
import tensorflow as tf



silver = [0.753, 0.753, 0.753]#{'color': [0.753, 0.753, 0.753], 'material': 
a_si = [0.600, 0.460, 0.357]
gold = [212/255, 175/255, 55/255]

si_material = open3d.visualization.Material('defaultLitSSR')
si_material.scalar_properties['metallic'] = 0.0
si_material.scalar_properties['roughness'] = 0.218
si_material.vector_properties['base_color'] = [*a_si, 1]
ag_material = open3d.visualization.Material('defaultLitSSR')
ag_material.scalar_properties['metallic'] = 1.0
ag_material.scalar_properties['roughness'] = 0.666
ag_material.vector_properties['base_color'] = [*silver, 1]

[Warning] Since Open3D 0.15, installing Open3D via conda is deprecated. Please re-install Open3D via: `pip install open3d -U`.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
filename = 'SCTF_Si_Ag_L256_x1.0_Th85.5_D30_N589824_1659041585'

grid, deposited, params = loadSparse('structures//' + filename + '.npz')
print(params)

grid_full = matrixFromSparse(grid)
grid_si = np.where(grid_full==1, 1, 0)
grid_ag = np.where(grid_full==2, 1, 0)

[{'L': 256, 'theta': 85.5, 'phi': 0, 'H': 64, 'D': 10, 'turns': 0.85, 'species': [1], 'weights': array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.2       , -0.30000001,  3.        ]]), 'repetition': 524288, 'time': 3105.510812997818}, {'L': 256, 'theta': 85.5, 'phi': 306.0, 'H': 96, 'D': 30, 'turns': 0.15, 'species': [2], 'weights': array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.2       , -0.30000001,  3.        ]]), 'repetition': 65536, 'time': 659.2555365562439}]


# Open3D works better

In [2]:
filename = 'STF_Si_Ag_L768_x1.0_Th85.5_D60_N9437184_1660238350'
filename = 'SCTF_Si_Ag_L256_x1.0_Th85.5_D30_N589824_1659041585'
si_meshes, ag_meshes = reconstructSiAgMeshColumns(filename, start=6, color_by_column=True, remove_floating=True)
open3d.visualization.draw_geometries([*ag_meshes, *si_meshes])

[{'L': 256, 'theta': 85.5, 'phi': 0, 'H': 64, 'D': 10, 'turns': 0.85, 'species': [1], 'weights': array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.2       , -0.30000001,  3.        ]]), 'repetition': 524288, 'time': 3105.510812997818}, {'L': 256, 'theta': 85.5, 'phi': 306.0, 'H': 96, 'D': 30, 'turns': 0.15, 'species': [2], 'weights': array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.2       , -0.30000001,  3.        ]]), 'repetition': 65536, 'time': 659.2555365562439}]
Disconnected floating components.
452 Ag regions:
	75 able to mesh; 377 unable for 4122 points left out.
190 Si regions:
	94 able to mesh; 96 unable for 1052 points left out.
Created surface meshes.


In [7]:
''' Clouds all at once '''
filename = 'STF_Si_Ag_L768_x1.0_Th85.5_D60_N9437184_1660238350'
si_mesh, ag_mesh = modeling.reconstructSiAgMeshesFull('SCTF_Si_Ag_L256_x1.0_Th85.5_D30_N589824_1659041585')
open3d.visualization.draw_geometries([ag_mesh, si_mesh])

Disconnected floating components.
Hollowed components.
Created point clouds.
Created surface meshes.


In [93]:
#open3d.visualization.draw_geometries([si_meshes[-1]])
#open3d.visualization.draw_geometries([*si_meshes, *ag_meshes, *si_comps, *ag_comps])
nor_list = []
van_list = []
ean_list = []
sin_list = []
els_list = []
for i in range(len(ag_meshes)):
    ag_meshes[i].remove_degenerate_triangles()
    if not ag_meshes[i].is_orientable():
        ag_meshes[i].paint_uniform_color([1, 0.5, 0.5])
        nor_list.append(i)
    if not ag_meshes[i].is_vertex_manifold():
        ag_meshes[i].paint_uniform_color([0.5, 1, 1]) # teal
        van_list.append(i)
    if not ag_meshes[i].is_edge_manifold():
        ag_meshes[i].paint_uniform_color([1, 0.5, 1]) # purple
        ean_list.append(i)
    if ag_meshes[i].is_self_intersecting():
        ag_meshes[i].paint_uniform_color([0, 1, 0.5]) # 
        sin_list.append(i)
    #else:
    #    ag_meshes[i].paint_uniform_color(silver)
    #    els_list.append(i)
print("Out of {}:\n\t{} not orientable\n\t{} are not vertex_manifold\n\t{} are not edge manifold\n\t{} are self intersecting\n\t{} else".
      format(len(ag_meshes), len(nor_list), len(van_list), len(ean_list), len(sin_list), len(els_list)))
open3d.visualization.draw_geometries([*si_meshes[:-1], *ag_meshes])

Out of 125:
	43 not orientable
	4 are not vertex_manifold
	43 are not edge manifold
	0 are self intersecting
	0 else


In [111]:
index = 1
lst = van_list
mesh = ag_meshes[lst[index]]
edges = mesh.get_non_manifold_edges(allow_boundary_edges=False)
print(np.asarray(edges))
line = open3d.geometry.LineSet(points=mesh.vertices, lines=edges)
line.colors = open3d.utility.Vector3dVector([[0,1,0]])
mesh.vertex_colors[edges[0][0]] = np.array([0, 1, 0])
mesh.vertex_colors[edges[0][1]] = np.array([0, 1, 0])
print(lst[index])
open3d.visualization.draw_geometries([ag_comps[lst[index]], line])

[[180 182]]
19


In [ ]:
render = open3d.visualization.rendering.OffscreenRenderer(1024, 1024)
silvermat = open3d.visualization.rendering.MaterialRecord()
silvermat.base_color = [*gold, 1]
silvermat.shader = 'defaultLit'
silvermat.base_metallic = 2
siliconmat = open3d.visualization.rendering.MaterialRecord()
siliconmat.base_color = [*a_si, 1]
siliconmat.shader = 'defaultLit'
siliconmat.base_roughness = 16

i = 0
for m in si_meshes:
    if m != si_meshes[-1]:
        render.scene.add_geometry("Si_{}".format(i), m, siliconmat)
        i += 1
i = 0
for m in ag_meshes:
    render.scene.add_geometry("Ag_{}".format(i), m, silvermat)
    i += 1
render.setup_camera(40, [384, 384, 0], [-384, 384, 512], [0, 0, 1])
render.scene.scene.set_sun_light([0, -0.5, -1], [1, 1, 1], 7500)
render.scene.scene.enable_sun_light(True)
render.scene.show_axes(False)
#render.set_clear_color([0,0,0,1])
img = render.render_to_image()
open3d.io.write_image("models/" + filename + "_front.png", img)
render.setup_camera(40, [384, 384, 0], [0, 0, 512], [0, 0, 1])
img = render.render_to_image()
open3d.io.write_image("models/" + filename + "_diagonal.png", img)
render.setup_camera(40, [384, 384, 0], [384, 384, 768], [0, 0, 1])
img = render.render_to_image()
open3d.io.write_image("models/" + filename + "_top.png", img)

In [ ]:
vis = open3d.visualization.VisualizerWithEditing()
vis.create_window()
for c in si_meshes:
    vis.add_geometry(c)
while not vis.is_full_screen():
    vis.poll_events()
    vis.update_renderer()
vis.destroy_window()

In [ ]:
vis.destroy_window()

In [89]:
# combine clouds and meshes
MasterSiCloud = open3d.geometry.PointCloud()
MasterAgCloud = open3d.geometry.PointCloud()
MasterSiMesh = open3d.geometry.TriangleMesh()
MasterAgMesh = open3d.geometry.TriangleMesh()
for c in si_comps:
    MasterSiCloud += c
for c in ag_comps:
    MasterAgCloud += c
for c in si_meshes:
    MasterSiMesh += c
for c in ag_meshes:
    MasterAgMesh += c
MasterCloud = MasterSiCloud + MasterAgCloud
MasterMesh = MasterSiMesh + MasterAgMesh

# save clouds and meshes
open3d.io.write_point_cloud('models/' + filename + '.pcd', MasterCloud)
open3d.io.write_triangle_mesh('models/' + filename + '.ply', MasterMesh)
open3d.io.write_point_cloud('models/' + filename + '_Si.pcd', MasterSiCloud)
open3d.io.write_triangle_mesh('models/' + filename + '_Si.ply', MasterSiMesh)
open3d.io.write_point_cloud('models/' + filename + '_Ag.pcd', MasterAgCloud)
open3d.io.write_triangle_mesh('models/' + filename + '_Ag.ply', MasterAgMesh)

True

# PyVista

In [ ]:
''' Test entire surface '''

grid_fill = scipy.ndimage.binary_fill_holes(np.tile(np.where(grid_full > 0, 1, 0), (1,1,1)))
grid_surf = hollow(grid_fill)
root = remove_unanchored(grid_surf, grid_fill)
sroot = sparseFromMatrix(root)
point_cloud = pv.PolyData(sroot[:,0:3])
point_cloud['species'] = sroot[:,3]
#point_cloud['vectors'] = calc_normals(sroot[:,:3])
point_cloud.compute_normals()
point_cloud.plot(eye_dome_lighting=True)

In [ ]:
#volume = point_cloud.delaunay_3d(progress_bar=True)
#shell = volume.extract_geometry()
#shell.plot()

volume2 = point_cloud.reconstruct_surface(nbr_sz=10, sample_spacing=None, progress_bar=True)
shell2 = volume2.extract_geometry()
shell2.plot()

In [ ]:
''' Find connected regions '''

print(grid_full.shape)
grid_si = np.where(grid_full==1, 1, 0)
grid_ag = np.where(grid_full==2, 1, 0)
regions1, number_of_components1 = scipy.ndimage.label(np.tile(grid_si, (1, 1, 1)), np.ones((3,3,3)))
regions2, number_of_components2 = scipy.ndimage.label(np.tile(grid_ag, (1, 1, 1)), np.ones((3,3,3)))
regions2 = np.where(regions2>0, regions2 + number_of_components1, 0)
regions = regions1 + regions2
print(regions1.shape)
#print(regions1, number_of_components)

comps = np.zeros((grid.shape[0]))
for i in range(grid.shape[0]):
    p = grid[i,:]
    comps[i] = regions[p[2], p[1], p[0]]
point_cloud['comps'] = comps
point_cloud.plot(eye_dome_lighting=True)


#slices = np.zeros(grid_full)
#for i in range(grid_full.shape[0]):

In [ ]:
struct = np.zeros((3,3,3))
struct[1,1,:] = 1
struct[1,:,1] = 1
struct[:,1,1] = 1
print(struct)

grid_si_fill = scipy.ndimage.binary_fill_holes(np.tile(grid_si, (1,1,1)), structure=struct)
grid_ag_fill = scipy.ndimage.binary_fill_holes(np.tile(grid_ag, (1,1,1)), structure=struct)
grid_fill = scipy.ndimage.binary_fill_holes(np.tile(np.where(grid_full > 0, 1, 0), (1,1,1)), structure=struct)

#grid_si_fill = scipy.ndimage.binary_fill_holes(grid_si_fill, structure=struct)
#grid_ag_fill = scipy.ndimage.binary_fill_holes(grid_ag_fill, structure=struct)
#grid_fill = scipy.ndimage.binary_fill_holes(grid_fill, structure=struct)

#grid_fill = grid_fill[:,grid_fill.shape[1]//3:(grid_fill.shape[1]*2)//3, grid_fill.shape[2]//3:(grid_fill.shape[2]*2)//3]
print(grid_fill.shape)

grid_interior = copy.deepcopy(grid_fill)
for k in range(grid_full.shape[0]):
    for j in range(grid_full.shape[1]):
        for i in range(grid_full.shape[2]):
            if grid_full[k,j,i] != 1:
                continue
            if grid_full[k,(j-1)%grid_full.shape[1],i] == 0 or grid_full[k,j,(i-1)%grid_full.shape[2]] == 0 or grid_full[k,(j+1)%grid_full.shape[1],i] == 0 or grid_full[k,j,(i+1)%grid_full.shape[2]] == 0:
                continue
            if k > 0:
                if grid_full[k-1,j,i] == 0:
                    continue
            if k < grid_full.shape[0]-1:
                if grid_full[k+1,j,i] == 0:
                    continue
            grid_interior[k,j,i] = 0
hollow = sparseFromMatrix(grid_interior)
hollow_cloud = pv.PolyData(hollow[:,0:3])
#hollow_cloud['species'] = grid[:,3]
hollow_cloud.plot(eye_dome_lighting=True)
print(point_cloud.number_of_points, hollow_cloud.number_of_points)

In [ ]:
''' Remove disconnected components '''

regions, number_of_components = scipy.ndimage.label(grid_interior, structure=np.ones((3,3,3)))
print(regions.shape)
inv_regions = np.where(regions == 1, 1, 0)
print(grid_interior.shape, inv_regions.shape)
root = grid_interior * inv_regions
connected = sparseFromMatrix(root)
connected_cloud = pv.PolyData(connected[:,0:3])
#connected_cloud['species'] = grid[:,3]
connected_cloud.plot(eye_dome_lighting=True)
print(point_cloud.number_of_points, connected_cloud.number_of_points)


In [ ]:
''' Ray tracing to find points that can be removed '''
root_hollow = copy.deepcopy(root)
for k in range(root.shape[0]):
    for j in range(root.shape[1]):
        for i in range(root.shape[2]):
            if(root[k,j,i]) == 0:
                continue
            # x
            if (root[k,j,(i-2)%root.shape[2]] == 0 and root[k,j,(i-1)%root.shape[2]] == 0) or (root[k,j,(i+2)%root.shape[2]] == 0 and root[k,j,(i+1)%root.shape[2]] == 0):
                continue
            # y
            if (root[k,(j-2)%root.shape[1],i] == 0 and root[k,(j-1)%root.shape[1],i] == 0) or (root[k,(j+2)%root.shape[1],1] == 0 and root[k,(j+1)%root.shape[1],i] == 0):
                continue
            # vertical
            if k > 1 and root[k-1,j,i] == 0:
                if k > 2 and root[k-2,j,i] == 0:
                    continue
                continue
            # xy, same sign
            if (root[k,(j-2)%root.shape[1],(i-2)%root.shape[2]] == 0 and root[k,(j-1)%root.shape[1],(i-1)%root.shape[2]] == 0) or (root[k,(j+2)%root.shape[1],(i+2)%root.shape[2]] == 0 and root[k,(j+1)%root.shape[1],(i+1)%root.shape[2]] == 0):
                continue
            # xy, diff sign
            if (root[k,(j-2)%root.shape[1],(i+2)%root.shape[2]] == 0 and root[k,(j-1)%root.shape[1],(i+1)%root.shape[2]] == 0) or (root[k,(j+2)%root.shape[1],(i-2)%root.shape[2]] == 0 and root[k,(j+1)%root.shape[1],(i-1)%root.shape[2]] == 0):
                continue
            # diagonals with attitude (do later because annoying)
            #if k > 1

            # this point cannot be seen so just remove it
            root_hollow[k,j,i] = 0
    
los = sparseFromMatrix(root_hollow)
los_cloud = pv.PolyData(los[:,0:3])
#los_cloud['species'] = grid[:,3]
los_cloud.plot(eye_dome_lighting=True)
print(point_cloud.number_of_points, los_cloud.number_of_points)

In [ ]:
''' Try to create a mesh '''
#volume = los_cloud.delaunay_3d(progress_bar=True)
#shell = volume.extract_geometry()
#shell.plot()

In [ ]:
''' Maybe a surface from the points? '''
#volume2 = los_cloud.reconstruct_surface(nbr_sz=10, sample_spacing=None, progress_bar=True)
#shell2 = volume2.extract_geometry()
#shell2.plot()